In [ ]:
import sys
sys.path.append(r"C:\Users\anderegg\Documents\dev\OSPARC\osparc-simcore-ssh\packages\simcore-sdk\src")
sys.path.append(r"C:\Users\anderegg\Documents\dev\OSPARC\osparc-simcore-ssh\packages\s3wrapper\src")

import logging
logging.basicConfig(level=logging.WARNING)

In [ ]:
import os
os.environ["SIMCORE_NODE_UUID"] = "3a97f542-93c4-419c-b5c9-bcf9ff3ada7e"
os.environ["PIPELINE_NODE_ID"] = "42"

os.environ["POSTGRES_ENDPOINT"] = "localhost:5432"
os.environ["POSTGRES_USER"] = "simcore"
os.environ["POSTGRES_PASSWORD"] = "simcore"
os.environ["POSTGRES_DB"] = "simcoredb"

os.environ["S3_ENDPOINT"] = "localhost:9001"
os.environ["S3_ACCESS_KEY"] = "12345678"
os.environ["S3_SECRET_KEY"] = "12345678"
os.environ["S3_BUCKET_NAME"] = "simcore"


In [ ]:
from simcore_sdk.nodeports import PORTS

In [ ]:
print(len(PORTS.inputs))
print(PORTS.inputs[0])

In [ ]:
print(PORTS.inputs[0].get())


In [ ]:
print(PORTS.outputs[0])

In [ ]:
PORTS.outputs["out_1"].get()

In [ ]:
PORTS.outputs["out_1"].set("Hey this still works or NOTTTTTT crazy shit man!!")